# Next Purchase Date with ML - Routine Transactions

In this use case, we use machine learning approach to know the model's performance if trained with the routine transactions between the user and the item. We assume that routine transactions are when a user buys a certain item at least once a month. To avoid overfitting for the too-frequent transactions (e.g., buying an item every day), we choose two interactions of user and item with the biggest average interval of transactions.

## Load and Preprocess Data

We use more than 500k transaction data between users and items from the EPM database. The raw data still has some returning transactions with a negative amount, but we are only looking for buying transactions. Each transaction has a timestamp record daily. Because a user can buy the same item multiple times on the same day, we consider it a single data aggregating the sales quantity column.

In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [2]:
# Load Dataset
df = pd.read_csv('/kaggle/input/epm-prep/EPM.csv')
df = df.drop(['Unnamed: 0', 'principal_code'], axis=1)
df.head()

,trx_date,customer_name,ship_to_id,branch_code,item_code,item_desc,principal_desc,gross_sales_amount,sales_qty
0,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,KCFMB,CEFIXIME 100MG 50 KAPSUL,HEXPHARM (PHARMAMED),195000,3
1,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,CKCOA,KALCINOL N CREAM 5 GR,KALBE NIMITZ (PHARMAMED),28500,3
2,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TMFNB,METFORMIN HCL 200 TABLET,HEXPHARM (PHARMAMED),175000,5
3,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TBSVC,BRONSOLVAN 100 TABLET,HEXPHARM TSJ (PHARMAMED),35000,1
4,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TALNF,AMLODIPINE BESILATE 10MG,HEXPHARM (PHARMAMED),255000,3


In [3]:
# Filter negative transactions
df = df[(df['sales_qty'] > 0) & (df['gross_sales_amount'] > 0)]
df['trx_date'] = pd.to_datetime(df['trx_date'])
df.head()

,trx_date,customer_name,ship_to_id,branch_code,item_code,item_desc,principal_desc,gross_sales_amount,sales_qty
0,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,KCFMB,CEFIXIME 100MG 50 KAPSUL,HEXPHARM (PHARMAMED),195000,3
1,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,CKCOA,KALCINOL N CREAM 5 GR,KALBE NIMITZ (PHARMAMED),28500,3
2,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TMFNB,METFORMIN HCL 200 TABLET,HEXPHARM (PHARMAMED),175000,5
3,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TBSVC,BRONSOLVAN 100 TABLET,HEXPHARM TSJ (PHARMAMED),35000,1
4,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TALNF,AMLODIPINE BESILATE 10MG,HEXPHARM (PHARMAMED),255000,3


In [4]:
# Drop duplicate transactions
temp = df[['ship_to_id', 'item_code', 'trx_date', 'sales_qty']].groupby(['ship_to_id', 'item_code', 'trx_date']).sum().reset_index(drop=True)
df = df.drop_duplicates(['ship_to_id', 'item_code', 'trx_date']).reset_index(drop=True)
df['sales_qty'] = temp
df.head()

,trx_date,customer_name,ship_to_id,branch_code,item_code,item_desc,principal_desc,gross_sales_amount,sales_qty
0,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,KCFMB,CEFIXIME 100MG 50 KAPSUL,HEXPHARM (PHARMAMED),195000,1
1,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,CKCOA,KALCINOL N CREAM 5 GR,KALBE NIMITZ (PHARMAMED),28500,1
2,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TMFNB,METFORMIN HCL 200 TABLET,HEXPHARM (PHARMAMED),175000,1
3,2021-11-02,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TBSVC,BRONSOLVAN 100 TABLET,HEXPHARM TSJ (PHARMAMED),35000,1
4,2021-08-20,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TALNF,AMLODIPINE BESILATE 10MG,HEXPHARM (PHARMAMED),255000,2


## Preprocessing for Usecase

First, we filter the data to be in the range of 2022 and 2023 only. We assume that 2021 or before is still in the COVID period, so the data may not be accurate and useful for predicting the present and future conditions. Not all the features are used in this use case; we only focused on the sales quantity and timestamp column.

In the process of searching the selected routine transactions, we do these steps:
1. Choose the user that has transactions at least once every month.
2. From the selected users, choose the item from each user that has transactions at least once every month.
3. From the selected users and items, calculate the average timestamp interval between all consecutive transactions in each user-item interaction. (in a daily unit)
4. Choose the user-item interactions with the biggest average interval. Why? Because we don't want to consider the data to be quite robust, as it will be too easy for the model to learn and predict it.

In [5]:
# Get transactions after 2021
dfDate = df[df['trx_date'] > '2021-12-31'].reset_index(drop=True)
dfDate.head()

,trx_date,customer_name,ship_to_id,branch_code,item_code,item_desc,principal_desc,gross_sales_amount,sales_qty
0,2022-01-12,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,KCFMB,CEFIXIME 100MG 50 KAPSUL,HEXPHARM (PHARMAMED),195000,3
1,2022-01-19,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TPRGR,PROMAG TABLET,PT SAKAFARMA LABORATORIES (CHD),71092,1
2,2022-01-26,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TVIPA,VITAZYM PLUS 100 TABLET,KALBE NIMITZ (PHARMAMED),75000,3
3,2022-01-25,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TALOD,ALLOPURINOL 300 MG 100 TAB,HEXPHARM (PHARMAMED),126000,2
4,2022-01-06,JK1-AP. DEVITA_GROUP_NA,EPM_34950,JK1,TSPLB,SPASMINAL 100 TABLET,HEXPHARM (PHARMAMED),80000,6


In [6]:
# Create timestamp column
dfDate['timestamp'] = dfDate['trx_date'].apply(lambda x: x.timestamp())
dfDate['trx_date'] = dfDate['trx_date'].dt.strftime('%Y-%m-%d')
dfDate = dfDate[['ship_to_id', 'item_code', 'trx_date', 'sales_qty', 'timestamp']]
dfDate

,ship_to_id,item_code,trx_date,sales_qty,timestamp
0,EPM_34950,KCFMB,2022-01-12,3,1.641946e+09
1,EPM_34950,TPRGR,2022-01-19,1,1.642550e+09
2,EPM_34950,TVIPA,2022-01-26,3,1.643155e+09
3,EPM_34950,TALOD,2022-01-25,2,1.643069e+09
4,EPM_34950,TSPLB,2022-01-06,6,1.641427e+09
...,...,...,...,...,...
312541,EPM_4554507,BLMBN,2023-09-30,1,1.696032e+09
312542,EPM_4554507,BLCGR,2023-09-30,1,1.696032e+09
312543,EPM_4554507,BLCVS,2023-09-30,2,1.696032e+09
312544,EPM_4554507,BLCVC,2023-09-30,4,1.696032e+09


In [7]:
# Check if user has transactions in every month
users = dfDate['ship_to_id'].unique()
userAv = []
for u in users:
    dfUser = dfDate[dfDate['ship_to_id'] == u]
    ada = True
    for i in range(1, 22):
        if i <= 12:
            ln = len(dfUser[(dfUser['trx_date'] >= f'2022-{str(i).zfill(2)}-01') & 
                            (dfUser['trx_date'] <= f'2022-{str(i).zfill(2)}-31')])
        else:
            ln = len(dfUser[(dfUser['trx_date'] >= f'2023-{str(i - 12).zfill(2)}-01') & 
                            (dfUser['trx_date'] <= f'2023-{str(i - 12).zfill(2)}-31')])
        if ln == 0:
            ada = False
    if ada:
        userAv.append(u)
    
f"Total Users: {len(userAv)}"

'Total Users: 214'

In [8]:
# Check if user and item has transactions in every month
freqs = []
for u in userAv:
    dfUser = dfDate[dfDate['ship_to_id'] == u]
    items = dfUser['item_code'].unique()
    for it in items:
        dfUserItem = dfUser[dfUser['item_code'] == it]
        ada = True
        for i in range(1, 22):
            if i <= 12:
                ln = len(dfUserItem[(dfUserItem['trx_date'] >= f'2022-{str(i).zfill(2)}-01') & 
                                    (dfUserItem['trx_date'] <= f'2022-{str(i).zfill(2)}-31')])
            else:
                ln = len(dfUserItem[(dfUserItem['trx_date'] >= f'2023-{str(i - 12).zfill(2)}-01') & 
                                    (dfUserItem['trx_date'] <= f'2023-{str(i - 12).zfill(2)}-31')])
            if ln == 0:
                ada = False
        if ada:
            freqs.append((u, it))

f"Total Pairs of User and Item: {len(freqs)}"

'Total Pairs of User and Item: 255'

In [9]:
# Pair example
freqs[:5]

[('EPM_34971', 'TMPSD'),
 ('EPM_34971', 'TALNE'),
 ('EPM_34971', 'KLSPB'),
 ('EPM_34972', 'TALNF'),
 ('EPM_34972', 'TMFNB')]

In [10]:
# Check average interval transactions, and choose the biggest one
means = []
dfDate['trx_date'] = pd.to_datetime(dfDate['trx_date'])
for f in freqs:
    dfFreq = dfDate[(dfDate['ship_to_id'] == f[0]) & (dfDate['item_code'] == f[1])].sort_values(by='trx_date') \
                                                                                   .reset_index(drop=True)

    dfFreq['period'] = dfFreq['trx_date'].diff() \
                                         .apply(lambda x: x.days)[1:] \
                                         .reset_index(drop=True) 
    means.append(dfFreq['period'].describe()['mean'])

    if means[-1] >= 24:
        print(f)

sorted(means)[-10:]

('EPM_136080', 'TRGCA')
('EPM_3041843', 'CKPXB')


[20.233333333333334,
 21.0,
 21.0,
 21.392857142857142,
 21.928571428571427,
 22.0,
 22.653846153846153,
 23.615384615384617,
 24.0,
 24.833333333333332]

In [11]:
# Show sample data
lst = [('EPM_136080', 'TRGCA'), ('EPM_3041843', 'CKPXB')]
for f in lst:
    dfShow = dfDate[(dfDate['item_code'] == f[1]) & (dfDate['ship_to_id'] == f[0])].sort_values('trx_date').reset_index(drop=True)
    dfShow['period'] = dfShow['trx_date'].diff().apply(lambda x: x.days)[1:].reset_index(drop=True)
    dfTrain, dfTest = dfShow[:-1], dfShow[-1:]
    print(f)
    print(dfTrain)
    print('-----------------------------')

('EPM_136080', 'TRGCA')
    ship_to_id item_code   trx_date  sales_qty     timestamp  period
0   EPM_136080     TRGCA 2022-01-05          4  1.641341e+09     5.0
1   EPM_136080     TRGCA 2022-01-10          3  1.641773e+09    31.0
2   EPM_136080     TRGCA 2022-02-10          1  1.644451e+09    40.0
3   EPM_136080     TRGCA 2022-03-22          5  1.647907e+09    28.0
4   EPM_136080     TRGCA 2022-04-19          2  1.650326e+09    10.0
5   EPM_136080     TRGCA 2022-04-29         10  1.651190e+09     8.0
6   EPM_136080     TRGCA 2022-05-07          2  1.651882e+09    17.0
7   EPM_136080     TRGCA 2022-05-24          1  1.653350e+09    28.0
8   EPM_136080     TRGCA 2022-06-21          2  1.655770e+09    30.0
9   EPM_136080     TRGCA 2022-07-21          4  1.658362e+09    18.0
10  EPM_136080     TRGCA 2022-08-08          6  1.659917e+09    28.0
11  EPM_136080     TRGCA 2022-09-05          3  1.662336e+09     3.0
12  EPM_136080     TRGCA 2022-09-08         10  1.662595e+09    41.0
13  EPM_13

## Training Model

After choosing a specific user and item, we put those data into the model. We try three different models and compare the performance. The models are trained with 80% of the data and evaluated with the rest by the Mean Average Error and Mean Squared Error metrics.

In [12]:
for f in lst:
    
    dfShow = dfDate[(dfDate['item_code'] == f[1]) & (dfDate['ship_to_id'] == f[0])].sort_values('trx_date').reset_index(drop=True)
    dfShow['period'] = dfShow['trx_date'].diff().apply(lambda x: x.days)[1:].reset_index(drop=True)
    dfTrain, dfTest = dfShow[:-1], dfShow[-1:] # The last row doesnt have interval value for next purchase

    dfTest = dfTest.drop(['period', 'ship_to_id', 'item_code', 'trx_date'], axis=1)
    X = dfTrain.drop(['period', 'ship_to_id', 'item_code', 'trx_date'], axis=1)
    y = dfTrain['period']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Linear Regression
    modelLR = LinearRegression()
    modelLR.fit(X_train, y_train)
    y_pred = modelLR.predict(X_test)
    maeLR = mean_absolute_error(y_test, y_pred)
    mseLR = mean_squared_error(y_test, y_pred)

    # Random Forest
    modelRF = RandomForestRegressor(random_state=42)
    modelRF.fit(X_train, y_train)
    y_pred = modelRF.predict(X_test)
    maeRF = mean_absolute_error(y_test, y_pred)
    mseRF = mean_squared_error(y_test, y_pred)
    
    # XGBoost
    modelXGB = XGBRegressor()
    modelXGB.fit(X_train, y_train)
    y_pred = modelXGB.predict(X_test)
    maeXGB = mean_absolute_error(y_test, y_pred)
    mseXGB = mean_squared_error(y_test, y_pred)

    print("{:<12} {:<8} \nLinear Regression MAE: {:<5} MSE: {:<6} \nRandom Forest     MAE: {:<5} MSE: {:<6} \nXGBoost           MAE: {:<5} MSE: {:<6}\n".format(f[0], f[1], round(maeLR, 2), round(mseLR, 2), round(maeRF, 2), round(mseRF, 2), round(maeXGB, 2), round(mseXGB, 2)))

EPM_136080   TRGCA    
Linear Regression MAE: 19.31 MSE: 717.22 
Random Forest     MAE: 14.73 MSE: 301.46 
XGBoost           MAE: 15.01 MSE: 284.1 

EPM_3041843  CKPXB    
Linear Regression MAE: 7.23  MSE: 60.38  
Random Forest     MAE: 7.75  MSE: 102.05 
XGBoost           MAE: 11.51 MSE: 242.78

